# &#128214; Lab 5: Symbolic Execution

## &#128221; Exercise 1: Introduction to Symbolic Execution

### &#127919; Objective
To introduce you to the concept of symbolic execution and demonstrate how to implement it in Python for program analysis.

---

### &#128161; Background

Symbolic execution is a sophisticated program analysis technique that explores multiple execution paths in a program by treating its inputs as symbolic variables instead of concrete values. This extensive exploration is vital for various applications including bug discovery, test case generation, and security analysis.

Key Elements:
- **Symbol**: A variable used to symbolize an unknown value. During program execution, symbols replace concrete values.
  
- **Constraint**: A mathematical condition that is imposed on symbols. For instance, `x + 5 > 10` can be considered a constraint on the symbolic variable `x`.
  
- **Path Constraint**: A set of constraints that, when satisfied, make a particular execution path possible. These constraints accumulate at conditional branches like `if-else` statements in the program.

- **Symbolic Variable**: A special variable that takes on symbolic values during execution, aiding in the formation of constraints and path constraints.

- **Solver**: An essential component in symbolic execution that attempts to find values for symbolic variables that satisfy the path constraints. Solvers like Z3 can be used to verify if a particular path is feasible or to find specific inputs that can trigger a particular path.

The exercise will guide you through the development of a simplified symbolic execution engine that can handle arithmetic operations and conditional branches in Python code. This engine will not only generate but also solve path constraints using a solver, thereby offering an in-depth understanding of how different conditions affect program behavior. The solver's role is crucial here, as it helps in validating the feasibility of paths and can even be used for tasks like automatic test case generation.

---

### &#10145; Task

Implement a basic symbolic execution engine that will explore both branches of a given if-else statement in Python code, assuming symbolic variables for the inputs.

---

### &#128221; Instructions

1. Use the `ast.parse` function to parse a simple Python script with an `if-else` statement into an AST.
2. Create a `SymbolicExecutor` class that will visit each `ast.If` node and simulate the symbolic execution for both the true and false branches.
3. Apply your `SymbolicExecutor` class to the AST obtained in step 1 using the `visit` method.
4. Generate constraints for both the true and false branches of the `if-else` statement based on the conditions.
5. Output the constraints for both paths, effectively demonstrating symbolic execution.

==> For simplification, you may assume that the script only contains arithmetic operations (`+`, `-`, `*`, `/`) and simple `if-else` statements. Your symbolic execution engine should be able to generate constraints like `x + y > 10` or `x * 2 < y`.

This exercise will deepen your understanding of symbolic execution, enabling you to appreciate its potential for advanced program analysis tasks.

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation
- `graphviz`: a library to create directed graphs


In [ ]:
import ast
import graphviz

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
a = input()
x = a * 4
if x == 8:
  print("OK")
else:
  print("NOK")
print("END")
"""

&#9874; Utility function

We provide a utility function called `print_dict` that can be used to print an indented representation of a `dict` object in Python.

In [ ]:
import json

def print_dict(dictionary):
    """
    Prints a dictionary in a beautified and indented format.

    Parameters:
    dictionary: The dictionary to be printed.

    Returns:
    None
    """
    print(json.dumps(dictionary, indent=4))

### Utility Control Flow Graph class

&#128161; The following cell contains a utility class to build a Control Glow Graph. 
You have to use this class to build the control flow graph.

In [ ]:
class ControlFlowGraph:
    """
    A class representing a Control Flow Graph (CFG).

    Attributes:
    nodes: A list where each element is a statement.
    edges: A list of tuples representing edges between nodes, where each tuple contains a pair of nodes.

    Methods:
    add_node:
        Adds a new node with the given statement to the graph, returning the new node.
    add_edge:
        Adds an edge between the specified node indices to the graph.
    visualize:
        Prints a visualization of the graph to the console.
    to_dot:
        Returns a DOT-format string representing the graph.
    """

    def __init__(self):
        """
        Initializes a new instance of the ControlFlowGraph class, with empty nodes and edges.
        """
        self.nodes = []
        self.edges = []

    def add_node(self, statement):
        """
        Adds a new node with the given statement to the graph.

        Parameters:
        statement: The statement associated with the new node.

        Returns:
        The node
        """
        self.nodes.append(statement)
        return statement

    def add_edge(self, node1, node2):
        """
        Adds an edge between the specified nodes to the graph.

        Parameters:
        node1: The source node.
        node2: The target node.
        """
        if (node1, node2) not in self.edges:
            self.edges.append((node1, node2))

    def visualize(self):
        """
        Prints a visualization of the graph to the console.
        Each edge is printed as a line in the format 'source -> target'.
        """
        for edge in self.edges:
            source, target = edge
            print(f'{source} -> {target}')

    def to_dot(self):
        """
        Returns a DOT-format string representing the graph (for vizualization purposes).

        Returns:
        str: A string in DOT format.
        """
        dot_lines = ['digraph cfg {']
        stmt_to_index = {}
        for index, statement in enumerate(self.nodes):
            stmt_to_index[statement] = index
            dot_lines.append(f'    node{index} [label="{statement}"];')
        for edge in self.edges:
            source, target = edge
            dot_lines.append(f'    node{stmt_to_index[source]} -> node{stmt_to_index[target]};')
        dot_lines.append('}')
        return '\n'.join(dot_lines)

### Function to create the control flow graph 

&#128161; In the following cell you can use the `build_cfg` function from the previous labs to get a CFG from an AST.

In [ ]:
def build_cfg(node, cfg, parent=None):
    """
    Recursively builds a Control Flow Graph (CFG) from an Abstract Syntax Tree (AST) node, considering various types of statements including assignment, expression, conditional, loop, and module level statements.

    Parameters:
    node: The current node in the AST.
    cfg: An instance of ControlFlowGraph to which nodes and edges will be added.
    parent: A list of parent nodes from which edges to the current node will be drawn. 
    
    Returns:
    list: A list of current nodes which will act as parents for the next level of recursion.
    """

&#128161; In the following cell, you will build the CFG and vizualize it with graphviz.

### Utility mapping

&#128161; The following cell contains a utility mapping that you can use to translate any ast operator into its equivalent.

In [ ]:
ast_operator_map = {
    "Add": "+",
    "Sub": "-",
    "Mult": "*",
    "Div": "/",
    "FloorDiv": "//",
    "Mod": "%",
    "Pow": "**",
    "LShift": "<<",
    "RShift": ">>",
    "BitOr": "|",
    "BitXor": "^",
    "BitAnd": "&",
    "Eq": "==",
    "NotEq": "!=",
    "Lt": "<",
    "LtE": "<=",
    "Gt": ">",
    "GtE": ">=",
    "Is": "is",
    "IsNot": "is not",
    "In": "in",
    "NotIn": "not in"
}

&#128161; The aim of this exercise is to implement the `SymbolicExecution` class that will perform symbolic execution on a CFG.

#### Instructions:

1. **Class Initialization**: 
    - Implement the `__init__` method to initialize the class attributes.
        - `cfg`: Control Flow Graph that will be analyzed.
        - `symbolic_vars`: A dictionary to store the symbolic variables encountered during the execution.
        - `path_constraints`: A dictionary to store the path constraints for each branch in the CFG.

2. **Executing AST Nodes**: 
    - Implement the `execute` method. It should take an AST `node` and a list of `path_constraints` as parameters. The method should recursively execute the node symbolically and return a list of tuples, each containing the next node to visit and any added constraint.
    
3. **Analysis**: 
    - Implement the `analyze` method to perform symbolic execution on the CFG. This method should populate `path_constraints` with constraints for each branch in the CFG.

#### Requirements:

- Use Python's AST library for working with AST nodes.
- Utilize the type-checking mechanism (e.g., `isinstance()`) to handle different types of AST nodes like `Assign`, `BinOp`, `Name`, `Num`, `Call`, `If`, and `Expr`.

#### Tips:

- Make use of helper dictionaries to map Python AST operator names to symbolic operators.
- Utilize a worklist algorithm to traverse the CFG nodes.

In [ ]:
class SymbolicExecution:
    """
    A class to perform symbolic execution on a Control Flow Graph (CFG) with AST nodes.

    Attributes:
        cfg (ControlFlowGraph): The CFG to be analyzed.
    """

    def __init__(self, cfg):

    def execute(self, node, path_constraints):
        """
        Recursively execute an AST node symbolically.
    
        Parameters:
            node (ast.AST): The AST node to execute.
            path_constraints (list): The path constraints so far.
        
        Returns:
            list: A list of tuples, each containing the next node to visit and the added constraint, if any.
        """

    def analyze(self):
        """
        Perform symbolic execution on the CFG.
    
        Returns:
            dict: A dictionary containing path constraints for each branch.
        """

&#128161; In the following cell, you will instantiate an object of type `SymbolicExecution` with the CFG previously constructed as a parameter and call the `analyze()` method on it.

This should return the path constraints computed.

&#128161; You can use the `pretty_print_constraints` function below to print the constraints in a more beautiful way.

In [ ]:
def pretty_print_constraints(constraints_dict):
    """
    Beautify and print the constraints for each statement.

    Parameters:
        constraints_dict (dict): Dictionary containing the path constraints for each node.
    """
    for node, path_constraints in constraints_dict.items():
        or_clauses = []
        for constraint_list in path_constraints:
            and_clause = " AND ".join(constraint_list)
            or_clauses.append(f"({and_clause})")
        final_expression = " OR ".join(or_clauses)
        print(f"Node {node}: {final_expression}")


In [ ]:
pretty_print_constraints(path_constraints)

&#10067; Do the constraint align with what you would have done manually?

Can you see some room for improvement?

## &#128221; Exercise 2: Solving Path Constraints using Z3 Solver

### &#127919; Objective
To extend the symbolic execution engine by incorporating the Z3 solver, which allows you to solve the path constraints and generate potential test cases.

---

### &#128161; Background

After symbolic execution, you obtain a set of path constraints representing the conditions that must be met for each execution path. 
To find concrete values that satisfy these constraints, you can use a constraint solver. 
The Z3 solver is highly efficient for this purpose and is commonly used in program analysis tasks.

---

### &#10145; Task

Implement the constraint-solving logic to transform the symbolic path constraints into solvable equations using the Z3 solver. The aim is to extract meaningful test cases from these solved constraints.

---

### &#128221; Instructions

1. **Import Z3 Modules**: Import essential Z3 modules like `Solver, And, Or, Int, Not, sat`.

2. **Implement `to_z3_expr` Function**: 
    - This function will translate a constraint string into a Z3 expression.
    - Take a constraint string and a dictionary of Z3 variables as parameters.
    - Return the Z3 expression.

3. **Implement `solve_constraints` Function**: 
    - This function will take a dictionary of path constraints and a dictionary of symbolic variables as parameters.
    - Utilize the Z3 solver to find a model that satisfies these constraints.
    - Store the satisfying model for each node in a dictionary and return it.

4. **Constraints to Z3 Expressions**: 
    - Translate each symbolic constraint to a Z3 expression using `to_z3_expr`.
    - Combine multiple constraints using logical AND (`And`) and OR (`Or`) operators from Z3.

5. **Solver Operations**: 
    - Push and pop the solver context appropriately to avoid conflicts between different sets of constraints.

6. **Check and Store Solutions**: 
    - For each node, if a satisfying set of input values exists (`solver.check() == sat`), store the Z3 model in a dictionary.

7. **Return Results**: 
    - Finally, return the dictionary containing the Z3 solutions for each node.

By the end of this exercise, you should be able to run your symbolic execution engine on sample code, solve the resulting path constraints, and generate possible test cases based on the Z3 solutions.

Install Z3  if you haven't
```python
!pip3 install z3-solver
```

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*

In [ ]:
from z3 import Solver, And, Or, Int, Not, sat

&#128161; In the following cell, implement the `to_z3_expr` and the `solve_constraints` functions.

In [1]:
def to_z3_expr(constraint, vars_dict):
    """
    Translate a constraint string into a Z3 expression.

    Parameters:
        constraint (str): The constraint string.
        vars_dict (dict): Dictionary containing Z3 variables.

    Returns:
        z3.ExprRef: The Z3 expression.
    """

def solve_constraints(constraints_dict, symbolic_vars):
    """
    Solve the constraints for each statement using Z3.

    Parameters:
        constraints_dict (dict): The dictionary containing path constraints for each statement.
        symbolic_vars (dict): Dictionary containing symbolic variables and their values.
        
    Returns:
        dict: A dictionary containing the Z3 solutions for each node.
    """

#### Test your code

&#128161; In the following cell, you will call the `solve_constraints` with the `path_constraints` variable and the `symbolic_execution.symbolic_vars` field.

&#128161; Print the solutions

&#10067; Did the solver find any solution?

If no, why?

If yes, do they match your expectations? 